## Aggregating ACS, FCC, Chicago Community Area Data

###### Importing Libraries

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

First, we use the FCC IL Dec 2020 csv file (which can be found [here](https://us-fcc.app.box.com/v/IL-Dec2020-v1)). Since it is a large file and we only want data on Chicago, we filter for Cook County data only and then export it as a new csv file which is saved in the data folder.

In [ ]:
# FCC IL

#fcc_df = pd.read_csv("data/IL-Fixed-Dec2020-v1.csv",index_col=0,parse_dates=[0])

In [ ]:
#changing BlockCode column to string type

#fcc_df['BlockCode']=fcc_df['BlockCode'].astype(str)

In [ ]:
# Extracting state, county, tract, block numbers from BlockCode column
# IL state=17
# Cook County=031

#fcc_df['state'] = fcc_df['BlockCode'].str[:2]
#fcc_df['county'] = fcc_df['BlockCode'].str[2:5]
#fcc_df['tract'] = fcc_df['BlockCode'].str[5:11]
#fcc_df['block'] = fcc_df['BlockCode'].str[-4:]

In [ ]:
# Filtering for Cook County only 
# 763788 rows

#chi_fcc = fcc_df[(fcc_df.county == "031")]

In [ ]:
# dropping columns we don't need to make the file smaller

#chi_fcc=chi_fcc.drop(['FRN', 'DBAName', 'HoldingCompanyName', 'HocoNum','HocoFinal', 'TechCode'], axis=1)

In [ ]:
#export final dataframe to csv file

#chi_fcc.to_csv(r'data/chi_fcc.csv', index = False)

###### Importing Datasets 

In [47]:
# CHICAGO FCC

fcc_df = pd.read_csv("data/chi_fcc.csv",index_col=0,parse_dates=[0])

In [48]:
fcc_df.head(5)

,ProviderName,StateAbbr,BlockCode,Consumer,MaxAdDown,MaxAdUp,Business,state,county,tract,block
Provider_Id,,,,,,,,,,,
69644,"TOWERSTREAM, INC.",IL,170310207023015,0,0.0,0.0,1,17,31,20702,3015
69644,"TOWERSTREAM, INC.",IL,170310803004005,0,0.0,0.0,1,17,31,80300,4005
69644,"TOWERSTREAM, INC.",IL,170310810005003,0,0.0,0.0,1,17,31,81000,5003
69644,"TOWERSTREAM, INC.",IL,170310813003001,0,0.0,0.0,1,17,31,81300,3001
69644,"TOWERSTREAM, INC.",IL,170310815002006,0,0.0,0.0,1,17,31,81500,2006


In [20]:
# ACS

acs_df = pd.read_csv("data/acs5_aggregate.csv",index_col=0,parse_dates=[0])

In [21]:
acs_df.head(5)

,B28001_002E,B28002_004E,B28003_002E,B28003_003E,B28003_004E,B28003_005E,B28003_006E,state,county,tract
0,426,392,426,0,392,34,149,17,31,630200
1,1411,1242,1411,0,1242,169,133,17,31,580700
2,1068,917,1068,11,917,140,104,17,31,590600
3,1003,917,1003,0,917,86,81,17,31,600700
4,356,322,356,0,322,34,218,17,31,611900


In [33]:
# Chicago community areas 

tracts = pd.read_csv("data/chi_tracts.csv",index_col=0,parse_dates=[0])

In [36]:
tracts.head(5)

,SUMLEV,COUNTY,COUSUB,PLACE,GEOID2,CHGOCA,TRACT,NAME,Notes,Community Area Name
STUSAB,,,,,,,,,,
IL,80,31,14000,14000,1010100,1,10100,CA 01 Tract 010100,NaN,Rogers Park
IL,80,31,14000,14000,1010201,1,10201,CA 01 Tract 010201,NaN,Rogers Park
IL,80,31,14000,14000,1010202,1,10202,CA 01 Tract 010202,NaN,Rogers Park
IL,80,31,14000,14000,1010300,1,10300,CA 01 Tract 010300,NaN,Rogers Park
IL,80,31,14000,14000,1010400,1,10400,CA 01 Tract 010400,NaN,Rogers Park
